In [ ]:
#anlık tweetlerin elde edilmesi
import base64
import datetime

from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
import tweepy
import time
import json

consumer_key=""
consumer_secret=""
access_key=""
access_secret=""




file = open(r"C:\Users\ASUS\Desktop\corona3.txt",'a',encoding="utf-8")

class StreamListener(tweepy.StreamListener):
    
    batch_size=50
    tweets=[]
    tweet_batch_size=[]
    
 
    def on_status(self, status):
        
            
        #s1 = json.dumps(status)
        #tweet_data = json.loads(s1)
        tweet_data = json.dumps(status._json)
        tweet_data=json.loads(tweet_data)
        if "extended_tweet" in tweet_data:
            tweet = tweet_data['extended_tweet']['full_text']
            print(tweet)
            if 'RT' not in tweet:
                file.write(tweet+"\n")
                


if __name__=='__main__':


    # complete authorization and initialize API endpoint
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_key, access_secret)
    api = tweepy.API(auth,parser=tweepy.parsers.JSONParser())

    streamListener = StreamListener()
    stream = tweepy.Stream(auth=api.auth, listener=streamListener,tweet_mode='extended')

    tags = ['stayhome','stay home','covid',"corona","coronavirus","korona","covid-19","Covid19","Covid-19","Corona Virus","pandemic","pandemi","COVID-19"]
    stream.filter(track=tags,languages = ["tr"])

In [ ]:
#eski tweetlere ulşmak için bu kod eklenebilir
import GetOldTweets3 as got
text_query = 'pandemi'
since_date = '2020-01-01'
until_date = '2020-04-30'
count = 10000
tweetCriteria = got.manager.TweetCriteria().setQuerySearch(text_query).setSince(since_date).setUntil(until_date).setMaxTweets(count).setLang('tr')
tweets = got.manager.TweetManager.getTweets(tweetCriteria)
text_tweets = [[tweet.date, tweet.text] for tweet in tweets]

In [ ]:
#duygusınıfları için 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
import re
import warnings
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB






data=pd.read_excel(r'C:\Users\ASUS\Desktop\duyguetiketi.xlsx')
data.head()


In [ ]:
data["sentiment"].replace(1, value = "pozitif", inplace = True)
data["sentiment"].replace(-1, value = "negatif", inplace = True)
data["sentiment"].replace(0, value = "nötr", inplace = True)


In [ ]:
labels = Counter(data['sentiment']).keys()
sum_ = Counter(data['sentiment']).values()
df = pd.DataFrame(zip(labels,sum_), columns = ['sentiment', 'Toplam'])

In [ ]:
#etiketlerin görselleştirilmesi - çubuk grafiği
df.plot(x = 'sentiment' , y = 'Toplam',kind = 'bar', legend = False, grid = True, figsize = (15,5))
plt.title('Kategori Sayılarının Görselleştirilmesi', fontsize = 20)
plt.xlabel('Kategoriler', fontsize = 15)
plt.ylabel('Toplam', fontsize = 15);
#etiketlerin görselleştirilmesi - pasta grafiği
fig, ax = plt.subplots(figsize=(15, 10))
ax.pie(df.Toplam, labels =df.sentiment, autopct = '%1.2f%%',  startangle = 90 )
ax.axis('equal')
plt.show()

In [ ]:
#veriönişleme

WPT = nltk.WordPunctTokenizer()
stop_word_list = nltk.corpus.stopwords.words('turkish')


docs = data['tweets']
docs = docs.map(lambda x: re.sub('[,\.!?();:$%&#"]', '', x))
docs = docs.map(lambda x: x.lower())
docs = docs.map(lambda x: x.strip())

#stopwordleri kaldırmak için
def token(values):
    filtered_words = [word for word in values.split() if word not in stop_word_list]
    not_stopword_doc = " ".join(filtered_words)
    return not_stopword_doc

docs = docs.map(lambda x: token(x))
data['tweets'] = docs

print(data.head(20))

In [ ]:
data.groupby("sentiment").count()

In [ ]:
dataDoc = data['tweets'].values.tolist()
dataClass = data['sentiment'].values.tolist()

In [ ]:
#test ve train olarak ayırma 
x_train, x_test, y_train, y_test = train_test_split(dataDoc, dataClass, test_size = 0.2, random_state = 42)

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df=5)
x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

#tfidf 
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1,2), max_df=0.9, min_df = 5)

x_train_tfidf = tfidf_vectorizer.fit_transform(x_train)
x_test_tfidf = tfidf_vectorizer.transform(x_test)

In [ ]:
#OneVsRest LogisticRegression modeli¶
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.multiclass import OneVsRestClassifier

#logistic regresyon
model = OneVsRestClassifier(LogisticRegression(penalty = 'l2', C=1.0))
model.fit(x_train_tfidf, y_train)

print ("Logistic Regression Accuracy={}".format(accuracy_score(y_test, model.predict(x_test_tfidf))))
logisticpred = accuracy_score(y_test, model.predict(x_test_tfidf))


#stochastic gradient descent
model2 = OneVsRestClassifier(SGDClassifier(loss = 'hinge', penalty = 'elasticnet', max_iter = 5))
model2.fit(x_train_tfidf, y_train)
print ("SGD Accuracy={}".format(accuracy_score(y_test, model2.predict(x_test_tfidf))))
sgdpred = accuracy_score(y_test, model2.predict(x_test_tfidf))


#lineersvc
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline

SVC_pipeline = Pipeline([
                ('clf', OneVsRestClassifier(LinearSVC(), n_jobs=1)),
            ])
SVC_pipeline.fit(x_train_tfidf, y_train)

prediction = SVC_pipeline.predict(x_test_tfidf)
print('LineerSVC accuracy is {}'.format(accuracy_score(y_test, prediction)))

In [ ]:
#nelerin konuşulduğunun analizi için
import pandas as pd
df=pd.read_excel(r'C:\Users\ASUS\Desktop\kategoriketiketler.xlsx')
df.head()

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
max=80

X_train, X_test, y_train, y_test = train_test_split(df['Tweets'], df['Labels'], test_size=0.20,random_state = 8)
count_vect = CountVectorizer( binary=True)
X_train_counts = count_vect.fit_transform(X_train)
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
clf = MultinomialNB(alpha=0.1,fit_prior=False).fit(X_train_tfidf, y_train)
clf1=SVC().fit(X_train_tfidf, y_train)
test_features = count_vect.transform(X_test)
y_pred=clf.predict(test_features)


print(accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
target_names=['Siyaset','Sağlık','Dünya','Spor'] 
y_pred=clf.predict_proba(test_features)*100
pred_df=pd.DataFrame(y_pred,columns=target_names)

pred_df=pd.DataFrame(y_pred,columns=target_names)
print(pred_df.iloc[231,:])


#%40 olasılık değerinden yukarıda olan tweetleri getirir.  
sayac=0
max=0
col=''
pred_df=pd.DataFrame(y_pred,columns=target_names)
for i in range(len(pred_df)):
    for j in pred_df.columns:
        
        if pred_df.loc[i,j]>40:
            sayac=sayac+1    #threshold değeri 
            print(pred_df.loc[i,j])
            print(i)
            print(j)
            print(pred_df.iloc[i,:])

    sayac=0
            



In [ ]:
test_data=pd.read_excel(r"C:\Users\ASUS\Desktop\covid19tweetler.xlsx")
test_data_list=[]
for i in range(len(test_data)):
    test_data_list.append(test_data.iloc[i,0])
    
test_etiketsiz = count_vect.transform(test_data_list)
y_pred_etiketsiz=clf.predict_proba(test_etiketsiz)*100

pred_df_test=pd.DataFrame(y_pred_etiketsiz,columns=target_names)
print(pred_df_test)

In [ ]:
#yeni gözlem

sayac=0
max=0
ilk=0
ilkin_etiketi=''
col=''
toplam=0
xc=pd.DataFrame(test_data)
pred_df_test=pd.DataFrame(y_pred_etiketsiz,columns=target_names)
tweet=[]

def test_tweet(i,j,deger):
    #print("Tweets:",xc.iloc[i,0])
    tweet.append((xc.iloc[i,0],j,deger))

for i in range(len(xc)):
    for j in pred_df_test.columns:        
        if pred_df_test.loc[i,j]>40: #threshold değeri 
            sayac=sayac+1   
            if sayac==1:
                ilk=pred_df_test.loc[i,j]
                ilkin_etiketi=j
                toplam=toplam+1
                
                test_tweet(i,ilkin_etiketi,ilk)               
                #print("************************************************")
            if sayac>=2:                
                #print("İkincisinin Değeri:",pred_df_test.loc[i,j])
                #print("İkincisinin Etiketi",j)
                toplam=toplam+1
                test_tweet(i,j,pred_df_test.loc[i,j])
                #print("************************************************")    

    sayac=0

tweet=pd.DataFrame(tweet).to_excel(r'C:\Users\ASUS\Desktop\TekveÇiftetiketeatanantwitler.xlsx',encoding="latin-1")
print("Toplam:",toplam)  
#18100 adet tweetten 17726 tanesi en az 1 tane sınıfa gitmiştir.17726 tanesinin içinden de 1028 tane tweet ise çift etikete gitmiştir.
